In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 7.9MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.2 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates In

In [2]:
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import drive 
drive.mount('/content/drive')
import re

Mounted at /content/drive


# 함수화 된 알고리즘 선언

In [3]:
def to_str(num):
  if not (type(num) == str):
    num = round(num)
    num = str (num)
  return num

def str_to_set(str_list):
  str_set = set(str_list)
  if ('' in str_set):
    str_set.remove('')
  return (' ').join(str_set)

In [4]:
def drop_dup_by_song_id(data_frame):
  #중복행들만 모아서 그 인덱스 리스트로 for문 돌림
  for idx in set(data_frame[data_frame['번호'].duplicated()].index.tolist()):
    #중복데이터 제일 위로 몰아주는 작업
    #장르 합치기
    genre_list = []
    for genre in data_frame.loc[idx]['장르'].values.tolist():
      genre_list += genre.split(' ')
    data_frame.loc[idx]['장르'].iloc[0] = str_to_set(genre_list)

    #좋아요 큰값으로 넣기
    luv = data_frame.loc[idx]['좋아요'].values.tolist()
    data_frame.loc[idx]['좋아요'].iloc[0] = str(max(luv))

    #태그 중복제거 합쳐서 넣기
    tag_list = []
    for tag in data_frame.loc[idx]['태그'].values.tolist():
      tag_list += tag.split(' ')
    data_frame.loc[idx]['태그'].iloc[0] = str_to_set(tag_list)

    
    #가수번호가 없다면 가수번호를 넣어준다.
    artist_num_string = ''
    if data_frame['가수번호'].loc[idx].iloc[0] == '':
      artist_num_set = set()
      for artist_num in data_frame.loc[idx]['가수번호'].values.tolist():
        artist_num_set.add(artist_num)
        if ('' in artist_num_set):
          artist_num_set.remove('')
      if len(artist_num_set) == 0:
        artist_num_string = ''
      else:
        artist_num_string = list(artist_num_set)[0]
    data_frame.loc[idx] = data_frame.loc[idx].replace({'가수번호':''}, {'가수번호':artist_num_string})

  return data_frame.drop_duplicates(subset='번호', keep='first')

#기존 데이터 불러오기
> 불러오고 전처리도 진행해야 한다.

In [5]:
Base_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/data/Base_Song_meta.csv', index_col=0)
# 가수가 nan인 값은 가수가 없는 고정 동요, 클래식 등이다.
Base_df = Base_df.fillna('')
#str로 입력되어있는 번호가 float형으로 읽히기때문에 str로 바꿔준다
Base_df['번호'] = Base_df['번호'].apply(to_str)
#Base_df['가수번호'] = Base_df['가수번호'].apply(to_str)
#체크하기
print(Base_df.shape)

Base_df
#Base_df.isna().sum()
#type(Base_df['번호'].iloc[0])

(4553, 10)


,날짜,번호,앨범번호,앨범이름,가수,가수번호,노래제목,장르,좋아요,태그
index,,,,,,,,,,
6677,20210706,6677,2005,Reminiscence (Piano Solo Selection),Yuhki Kuramoto,3162,Lake Louise,뉴에이지,"9,281",#뉴에이지 #집중 #공부 #힐링
6694,20210706,6694,2006,Lake Misty Blue,Yuhki Kuramoto,3162,Lake Louis Ⅱ,뉴에이지,"3,361",#공부할_때_듣는_음악 #공부 #집중
9573,20210706,9573,781,소품집,플라워,100391,Endless,록/메탈,"43,159",#2000년 #싸이월드 #추억 #2000년대
14776,20210706,14776,829,Merry Christmas (Deluxe Anniversary Edition),Mariah Carey,2274,All I Want For Christmas Is You,R&B/Soul,"172,409",#캐럴 #감성 #팝송 #Pop
19807,20210706,19807,2393,Brown Eyes,브라운 아이즈,100742,벌써 일년,R&B/Soul,"64,140",#2000년 #싸이월드 #이별 #그리움
...,...,...,...,...,...,...,...,...,...,...
33584886,20210716,33584886,10629826,Candy,Ruben,1053946,Candy,POP,106,#세련된 #혼자 #라운지 #달닻 #트렌디 #기분전환 #카페 #팝 #감각 #분위기 #...
33590554,20210716,33590554,10631338,Famous,Mark Dohner,2634948,Famous,POP,31,#세련된 #라운지 #달닻 #트렌디 #기분전환 #카페 #팝 #감각 #분위기 #감각적
33595733,20210716,33595733,10632158,Ocean~,Kuwada,2242267,Ocean~,POP,5,#선선한 #Chillout #나른한 #Chill #빈티지 #여름밤 #인디락 #매장에...


## 백업해주기
> 혹시모를 사고를 대비해 백업해준다

In [6]:
Base_df.to_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/data/Base_Song_meta_back_up.csv')

#일간랭킹 합치기

##데이터 프레임 초기화(날자 바꿔야함)

In [8]:
#일간 랭킹 불러오기
daily_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/일간랭킹/20210714일간랭킹_모든항목.csv')
# 가수가 nan인 값은 가수가 없는 고정 동요, 클래식 등이다.

daily_df = daily_df.fillna('')
#str로 입력되어있는 번호가 float형으로 읽히기때문에 str로 바꿔준다
daily_df['번호'] = daily_df['번호'].apply(to_str)
daily_df['가수번호'] = daily_df['가수번호'].apply(to_str)
#멜론에서 hidden처리한 곡과, 안쓰는 인덱스 삭제
#daily_df = daily_df.drop(labels=1603)
daily_df = daily_df.drop(labels='Unnamed: 0', axis=1)

#노래 번호를 int화 시켜서 인덱스로 만들어줌
daily_df['index'] = daily_df['번호'].apply((lambda x: int(x)))
daily_df = daily_df.set_index('index').sort_index()

daily_df

,날짜,번호,앨범번호,앨범이름,가수,가수번호,노래제목,장르,좋아요,태그
index,,,,,,,,,,
6677,20210714,6677,2005,Reminiscence (Piano Solo Selection),Yuhki Kuramoto,3162,Lake Louise,뉴에이지,"9,304",#인기순위 #일간순위 #뉴에이지
6694,20210714,6694,2006,Lake Misty Blue,Yuhki Kuramoto,3162,Lake Louis Ⅱ,뉴에이지,"3,368",#인기순위 #일간순위 #뉴에이지
8588,20210714,8588,2169,가야송,숙명 가야금 연주단,100278,터키행진곡,국악,169,#인기순위 #일간순위 #국악
9573,20210714,9573,781,소품집,플라워,100391,Endless,록/메탈,"43,283",#인기순위 #일간순위 #록/메탈
14776,20210714,14776,829,Merry Christmas (Deluxe Anniversary Edition),Mariah Carey,2274,All I Want For Christmas Is You,R&B/Soul,"172,475",#인기순위 #일간순위 #R&B/Soul
...,...,...,...,...,...,...,...,...,...,...
106730193,20210714,106730193,5640611,Red,Taylor Swift,325867,Red,포크/블루스/컨트리,"1,531",#인기순위 #일간순위 #포크/블루스/컨트리
106730195,20210714,106730195,5640611,Red,Taylor Swift,325867,I Knew You Were Trouble.,포크/블루스/컨트리,"2,169",#인기순위 #일간순위 #포크/블루스/컨트리
106730197,20210714,106730197,5640611,Red,Taylor Swift,325867,22,포크/블루스/컨트리,"4,251",#인기순위 #일간순위 #포크/블루스/컨트리


##중복이 아닌 노래들 따로 모으기
> 중복이 아닌 노래는 태그및 장르를 업데이트해줘야 하기 때문에 따로 모아놔야 한다

In [9]:
non_dup_set = set()
daily_set = set(daily_df.index.tolist())
Base_set = set(Base_df.index.tolist())
non_dup_list = list(non_dup_set.union(daily_set - Base_set))
len(non_dup_list)

27

## 기존 데이터베이스와 합치기

In [10]:
Base_df = pd.concat([Base_df, daily_df], axis=0)

In [11]:
Base_df.shape

(7253, 10)

## 중복제거하기
> 같은 노래의 다른 정보는 모두 처음행에 몰아주고
나머지는 지운다.

In [12]:
#중복된값 체크
print(Base_df[Base_df['번호'].duplicated()].shape)
Base_df[Base_df['번호'].duplicated()].head(3)

(2673, 10)


,날짜,번호,앨범번호,앨범이름,가수,가수번호,노래제목,장르,좋아요,태그
index,,,,,,,,,,
6677,20210714,6677,2005,Reminiscence (Piano Solo Selection),Yuhki Kuramoto,3162,Lake Louise,뉴에이지,"9,304",#인기순위 #일간순위 #뉴에이지
6694,20210714,6694,2006,Lake Misty Blue,Yuhki Kuramoto,3162,Lake Louis Ⅱ,뉴에이지,"3,368",#인기순위 #일간순위 #뉴에이지
8588,20210714,8588,2169,가야송,숙명 가야금 연주단,100278,터키행진곡,국악,169,#인기순위 #일간순위 #국악


In [13]:
#가수번호가 없었기때문에 nan값이생겨 fillna로 그런값을 다시 지워준다.
Base_df = Base_df.fillna('')
Base_df = drop_dup_by_song_id(Base_df)
Base_df.shape

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

(4580, 10)

#멜론DJ파일 합치기

##데이터 초기화

In [17]:
#일간 DJ 플레이리스트 읽어오기
day = datetime.today().strftime('%Y%m%d')
daily_dj_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/일간멜론DJ/'+day+'일간멜론DJ.csv')
# 장르가 nan인것은 장르를 입력할때 입력하지 못했기 때문이다. 이는 이후 전처리를 통해 해결될것이다.
daily_dj_df = daily_dj_df.fillna('')
daily_dj_df['번호'] = daily_dj_df['번호'].apply(to_str)
daily_dj_df['가수번호'] = daily_dj_df['가수번호'].apply(to_str)
daily_dj_df = daily_dj_df.drop(labels='Unnamed: 0', axis=1)

daily_dj_df['index'] = daily_dj_df['번호'].apply((lambda x: int(x)))
daily_dj_df = daily_dj_df.set_index('index').sort_index()

daily_dj_df

,날짜,번호,앨범번호,앨범이름,가수,가수번호,노래제목,장르,좋아요,태그,멜론DJ,플레이리스트,플레이리스트_번호,플레이리스트_태그
index,,,,,,,,,,,,,,
19541,20210716,19541,2374,Miss E ...So Addictive,Missy Elliott,35127,4 My People,,85,#힙하우스 #해외일렉트로니카 #1990 #해외랩힙합 #클럽 #힙합 #electron...,ELECTRONICA마스터,"하우스 음악과 힙합의 만남, 힙 하우스",433154797,#힙하우스 #해외일렉트로니카 #1990 #해외랩힙합 #클럽 #힙합 #electron...
37003,20210716,37003,3457,Roller Coaster,롤러 코스터,100195,어디 있나요,,522,#감성 #밴드 #빈티지 #새벽 #밤 #드라이브 #잔잔한 #잠들기전 #기분전환 #퇴근길,Z_aYn,NEW 취향 가득 눌러 담은 밴드 노래,495659065,#감성 #밴드 #빈티지 #새벽 #밤 #드라이브 #잔잔한 #잠들기전 #기분전환 #퇴근길
37004,20210716,37004,3457,Roller Coaster,롤러 코스터,100195,습관 (Bye Bye),,"33,354",#감성 #밴드 #빈티지 #새벽 #밤 #드라이브 #잔잔한 #잠들기전 #기분전환 #퇴근길,Z_aYn,NEW 취향 가득 눌러 담은 밴드 노래,495659065,#감성 #밴드 #빈티지 #새벽 #밤 #드라이브 #잔잔한 #잠들기전 #기분전환 #퇴근길
37101,20210716,37101,3449,일상다반사(日常茶飯事),롤러 코스터,100195,Love Virus,,"5,246",#감성 #밴드 #빈티지 #새벽 #밤 #드라이브 #잔잔한 #잠들기전 #기분전환 #퇴근길,Z_aYn,NEW 취향 가득 눌러 담은 밴드 노래,495659065,#감성 #밴드 #빈티지 #새벽 #밤 #드라이브 #잔잔한 #잠들기전 #기분전환 #퇴근길
38617,20210716,38617,3019,거짓말도 보여요,김현철,1424,그대 내게 행복을 주는 사람,,"1,718",#리메이크 #발라드 #여름 #바다 #상쾌한 #시원한 #명곡 #기분전환 #추억,대중가요마스터,상쾌한 리메이크 발라드 듣기,495080094,#리메이크 #발라드 #여름 #바다 #상쾌한 #시원한 #명곡 #기분전환 #추억
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33669679,20210716,33669679,10648820,Orange Drive,서울다반사,2935580,Orange Drive,,123,#국내록 #인디밴드 #Rock #록 #인디 #신예밴드 #모던록 #루키 #국내밴드,분휘기있게,더욱더 유명해져야 하는 국내 밴드,455957078,#국내록 #인디밴드 #Rock #록 #인디 #신예밴드 #모던록 #루키 #국내밴드
33669679,20210716,33669679,10648820,Orange Drive,서울다반사,2935580,Orange Drive,,123,#국내록 #인디밴드 #Rock #록 #인디 #신예밴드 #모던록 #루키 #국내밴드,분휘기있게,더욱더 유명해져야 하는 국내 밴드,455957078,#국내록 #인디밴드 #Rock #록 #인디 #신예밴드 #모던록 #루키 #국내밴드
101339662,20210716,101339662,5044264,(Why Do You Think You're So) Special?,The Honeymoon,827674,(Why Do You Think You're So) Special?,,23,#청춘 #하이틴 #기분좋은 #산뜻한 #얼터너티브 #락 #빈티지 #기분전환,공중정원.,빈티지한 하이틴 재질 Alternative pop,493536616,#청춘 #하이틴 #기분좋은 #산뜻한 #얼터너티브 #락 #빈티지 #기분전환


## 플레이리스트 관련정보는 뺴기

In [18]:
df_songs_df = daily_dj_df[['날짜', '번호', '앨범번호', '앨범이름', '가수', '노래제목', '장르', '좋아요', '태그']]
df_songs_df

,날짜,번호,앨범번호,앨범이름,가수,노래제목,장르,좋아요,태그
index,,,,,,,,,
19541,20210716,19541,2374,Miss E ...So Addictive,Missy Elliott,4 My People,,85,#힙하우스 #해외일렉트로니카 #1990 #해외랩힙합 #클럽 #힙합 #electron...
37003,20210716,37003,3457,Roller Coaster,롤러 코스터,어디 있나요,,522,#감성 #밴드 #빈티지 #새벽 #밤 #드라이브 #잔잔한 #잠들기전 #기분전환 #퇴근길
37004,20210716,37004,3457,Roller Coaster,롤러 코스터,습관 (Bye Bye),,"33,354",#감성 #밴드 #빈티지 #새벽 #밤 #드라이브 #잔잔한 #잠들기전 #기분전환 #퇴근길
37101,20210716,37101,3449,일상다반사(日常茶飯事),롤러 코스터,Love Virus,,"5,246",#감성 #밴드 #빈티지 #새벽 #밤 #드라이브 #잔잔한 #잠들기전 #기분전환 #퇴근길
38617,20210716,38617,3019,거짓말도 보여요,김현철,그대 내게 행복을 주는 사람,,"1,718",#리메이크 #발라드 #여름 #바다 #상쾌한 #시원한 #명곡 #기분전환 #추억
...,...,...,...,...,...,...,...,...,...
33669679,20210716,33669679,10648820,Orange Drive,서울다반사,Orange Drive,,123,#국내록 #인디밴드 #Rock #록 #인디 #신예밴드 #모던록 #루키 #국내밴드
33669679,20210716,33669679,10648820,Orange Drive,서울다반사,Orange Drive,,123,#국내록 #인디밴드 #Rock #록 #인디 #신예밴드 #모던록 #루키 #국내밴드
101339662,20210716,101339662,5044264,(Why Do You Think You're So) Special?,The Honeymoon,(Why Do You Think You're So) Special?,,23,#청춘 #하이틴 #기분좋은 #산뜻한 #얼터너티브 #락 #빈티지 #기분전환


##중복이 아닌 노래들 따로 모으기

In [19]:
dailydj_set = set(daily_dj_df.index.tolist())
Base_set = set(Base_df.index.tolist())
non_dup_list += list(non_dup_set.union(dailydj_set - Base_set))

non_dup_set = set(non_dup_list)
non_dup_list = list(non_dup_set)
len(non_dup_list)

27

## 데이터 프레임 합치기

In [ ]:
Base_df = pd.concat([Base_df, df_songs_df], axis=0)
Base_df.shape

(5799, 10)

##중복값 확인하기

In [ ]:
Base_df[Base_df.duplicated()]

,날짜,번호,앨범번호,앨범이름,가수,가수번호,노래제목,장르,좋아요,태그
index,,,,,,,,,,
931963,20210716,931963,308210,새드무비 OST,장필순,NaN,10월,,272,#OST #여름 #더위 #사랑 #설렘 #기분전환 #기분좋은 #드라이브 #출근길 #퇴근길
1173836,20210716,1173836,316938,궁(宮) 외전 OST,하울,NaN,사랑에 빠지다 (Duet.루나),,329,#OST #여름 #더위 #사랑 #설렘 #기분전환 #기분좋은 #드라이브 #출근길 #퇴근길
1175028,20210716,1175028,317028,토마토 OST,이형석,NaN,내게 온 사랑,,341,#OST #여름 #더위 #사랑 #설렘 #기분전환 #기분좋은 #드라이브 #출근길 #퇴근길
1526398,20210716,1526398,336197,러빙유 OST,Iris,NaN,Loving You,,623,#OST #여름 #더위 #사랑 #설렘 #기분전환 #기분좋은 #드라이브 #출근길 #퇴근길
1571649,20210716,1571649,344981,"Fish, Out From Water",NY물고기,NaN,같은 삶,,66,#애정 #밴드 #가수 #명곡 #인디 #모던락 #인디밴드 #싸이키델릭 #기타 #어쿠스틱
...,...,...,...,...,...,...,...,...,...,...
33569070,20210716,33569070,10625167,No Time To Stress,"Iggi Kelly , Louis III , Embody",NaN,No Time To Stress,,86,#트렌디 #기분전환 #세련된 #감각적 #드라이브 #여행 #아침
33571283,20210716,33571283,10625785,For My Future Past,"Kat Saul , Midsplit",NaN,Sleep Cycle,,11,#트렌디 #기분전환 #세련된 #감각적 #드라이브 #여행 #아침
33584125,20210716,33584125,10629537,Nothing But Love,"Rob Late , Tianda",NaN,Nothing But Love,,56,#트렌디 #기분전환 #세련된 #감각적 #드라이브 #여행 #아침


##중복값 지우기

In [ ]:
Base_df = drop_dup_by_song_id(Base_df)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

In [15]:
Base_df.shape

(4580, 10)

#완성된 노래 리스트 저장
> 크롤링에 문제가 생길 수 있으니 먼저 한번 저장해준다.

In [20]:
#테스트용 백업데이터에 저장
Base_df.to_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/data/Base_Song_meta_back_up.csv')

# 중복이 아닌 새로운 노래, 데이터 추가작업하기

In [21]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

main_url = 'https://www.melon.com/song/detail.htm?songId='

#장르 리스트화
genres = ['발라드', '댄스', '랩/힙합', 'R&B/Soul', '인디음악', '록/메탈', '트로트', '포크/블루스', 'POP', '록/메탈', '일렉트로니카', '랩/힙합', 'R&B/Soul', '포크/블루스/컨트리', 'OST', '클래식', '재즈', '뉴에이지', 'JPOP', '월드뮤직', 'CCM', '어린이/태교', '종교음악', '국악']

#새로운 노래 set -> list화
idx_list = non_dup_list

#에러는 따로 모아놓자
err_list = []

for idx in idx_list:
  url = main_url + Base_df['번호'].loc[idx]
  driver.get(url)
  time.sleep(3)

  try:
    body = driver.find_element_by_css_selector('body')
    res = driver.page_source
    soup = bs(res, "html.parser")

    #장르 업데이트
    print(Base_df['장르'].loc[idx], Base_df['노래제목'].loc[idx], Base_df['태그'].loc[idx])
    base_genre = Base_df['장르'].loc[idx].split(' ')

    dl_list = soup.find('dl', {'class':'list'})
    dl_list_text = dl_list.text
    dl_list_text = dl_list_text.split('\n')
    genre_index = dl_list_text.index('장르')
    genre_set = set([dl_list_text[genre_index+1]])
    genre_set.add(Base_df['장르'].loc[idx])
    if ('' in genre_set):
      genre_set.remove('')
    Base_df['장르'].loc[idx] = (' ').join(list(genre_set))

    #노래에 추천 태그 긁어오기
    div_service_list_album = soup.find('div', {'class':'service_list_album'})
    tag_ul = div_service_list_album.find('ul')
    tag_div = tag_ul.find_all('div', {'class':'tag_list'})
    #긁어온 태그 리스트화
    tag_list = []
    for div in tag_div:
      tag_list += div.text.replace('\n', ' ')[1:-1].split(' ')
    #기존에 있던 태그 리스트화
    tag_list_ori = Base_df['태그'].loc[idx].split(' ')
    #기존태그 + 새로운태그 and set화하여 중복 제거 및 공백태그제거
    tag_list += tag_list_ori
    tag_set = set(tag_list)
    if '' in tag_set:
      tag_set.remove('')
    Base_df['태그'].loc[idx] = (' ').join(tag_set)

    #진행사항 표시
    print('->', Base_df['장르'].loc[idx], Base_df['노래제목'].loc[idx], Base_df['태그'].loc[idx], idx_list.index(idx)+1, '/', len(idx_list))
  except:
    print('err')
    err_list.append(idx)
    continue
#파일로 저장

print('error num', len(err_list))
Base_df.to_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/data/Base_Song_meta.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


종교음악 광명진언 216독 · 이산혜연선사 발원문 #인기순위 #일간순위 #종교음악
err
R&B/Soul Amtrak #인기순위 #일간순위 #R&B/Soul
-> R&B/Soul Amtrak #알앤비탑백 #R&B/Soul #베드룸팝 #멜랑콜리 #일간순위 #알앤비 #인기순위 2 / 27


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


랩/힙합 삐딱하게 (Crooked) #인기순위 #일간순위 #랩/힙합
-> 랩/힙합 삐딱하게 (Crooked) #좋아요10만 #숨듣명 #회상 #일간순위 #노동요 #랩/힙합 #인기순위 3 / 27
월드뮤직 Pineapple Wine #인기순위 #일간순위 #월드뮤직
-> 월드뮤직 월드뮤직, 레게 Pineapple Wine #월드뮤직 #팝 #휴양지 #tropical #내적댄스 #일간순위 #인기순위 4 / 27
록/메탈 잘 지내자, 우리 #인기순위 #일간순위 #록/메탈
-> 인디음악, 록/메탈 록/메탈 잘 지내자, 우리 #인디탑백 #인디 #록/메탈 #일간순위 #감성 #까페 #인기순위 5 / 27
R&B/Soul whoops #인기순위 #일간순위 #R&B/Soul
-> R&B/Soul whoops #R&B/Soul #오후 #팝 #고막남친 #일간순위 #노동요 #인기순위 6 / 27
댄스 Summertime (Feat. 김효은) #인기순위 #일간순위 #댄스
-> 댄스 Summertime (Feat. 김효은) #댄스 #국내힙합 #힙합 #일간순위 #인기순위 #댄스탑백 7 / 27
클래식 드보르작 : 유모레스크 작품 101의 제7번 #인기순위 #일간순위 #클래식
-> 클래식 드보르작 : 유모레스크 작품 101의 제7번 #기분전환 #클래식 #일간순위 #인기순위 #봄 8 / 27
발라드 큰일이다 #인기순위 #일간순위 #발라드
-> 발라드 큰일이다 #싸이월드 #이별 #일간순위 #2000년 #발라드 #감성 #인기순위 9 / 27
일렉트로니카 Post Malone (feat. RANI) #인기순위 #일간순위 #일렉트로니카
-> 일렉트로니카 Post Malone (feat. RANI) #기분전환 #일렉트로니카 #일간순위 #드라이브 #헬스 #운동 #인기순위 10 / 27
클래식 드보르작 : 유모레스크 #인기순위 #일간순위 #클래식
-> 클래식 드보르작 : 유모레스크 #노동요 #일할때 #클래식 #일간순위 #휴식 #작업할때 #인기순위 11 / 27
댄스 Summer Taste #인기순위 #일간순

#에러처리

In [22]:
Base_df.loc[err_list]

,날짜,번호,앨범번호,앨범이름,가수,가수번호,노래제목,장르,좋아요,태그
index,,,,,,,,,,
32116864,20210714,32116864,10342307,혜봉스님 불경 13집(소원성취 진언 2),혜봉스님,2637644,광명진언 216독 · 이산혜연선사 발원문,불교 종교음악,35,#인기순위 #일간순위 #종교음악


> 현재는 들어가있는 정보는 다 있는것같다. 패스하도록 한다.

In [23]:
Base_df.to_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/data/Base_Song_meta.csv')

In [24]:
Base_df

,날짜,번호,앨범번호,앨범이름,가수,가수번호,노래제목,장르,좋아요,태그
index,,,,,,,,,,
6677,20210706,6677,2005,Reminiscence (Piano Solo Selection),Yuhki Kuramoto,3162,Lake Louise,뉴에이지,"9,281",#뉴에이지 #집중 #공부 #힐링
6694,20210706,6694,2006,Lake Misty Blue,Yuhki Kuramoto,3162,Lake Louis Ⅱ,뉴에이지,"3,361",#공부할_때_듣는_음악 #공부 #집중
9573,20210706,9573,781,소품집,플라워,100391,Endless,록/메탈,"43,159",#2000년 #싸이월드 #추억 #2000년대
14776,20210706,14776,829,Merry Christmas (Deluxe Anniversary Edition),Mariah Carey,2274,All I Want For Christmas Is You,R&B/Soul,"172,409",#캐럴 #감성 #팝송 #Pop
19807,20210706,19807,2393,Brown Eyes,브라운 아이즈,100742,벌써 일년,R&B/Soul,"64,140",#2000년 #싸이월드 #이별 #그리움
...,...,...,...,...,...,...,...,...,...,...
33550581,20210714,33550581,10621770,pretty when i cry,"Tim Legend , Shanin Blake",957238 2942469,pretty when i cry,일렉트로니카,33,#카페 #팝 #일렉트로니카 #일간순위 #일렉탑백 #일렉 #인기순위
33612254,20210714,33612254,10634810,Day By Day,Miller Blue,2556210,Day By Day,R&B/Soul,70,#알앤비탑백 #R&B/Soul #일간순위 #트렌디 #알앤비 #인기순위
33680289,20210714,33680289,10651215,"잘 지내자, 우리",최유리,2645970,"잘 지내자, 우리","인디음악, 록/메탈 록/메탈","1,561",#인디탑백 #인디 #록/메탈 #일간순위 #감성 #까페 #인기순위


#가수번호가 없는 노래 살펴보기

In [25]:
Base_df = Base_df.fillna('')
non_artist_num = Base_df[(Base_df['가수번호']=='') & (Base_df['가수']!='')].index.tolist()
len(non_artist_num)

0

In [26]:
Base_df.loc[non_artist_num[0]]

IndexError: ignored

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

main_url = 'https://www.melon.com/song/detail.htm?songId='

#장르 리스트화
genres = ['발라드', '댄스', '랩/힙합', 'R&B/Soul', '인디음악', '록/메탈', '트로트', '포크/블루스', 'POP', '록/메탈', '일렉트로니카', '랩/힙합', 'R&B/Soul', '포크/블루스/컨트리', 'OST', '클래식', '재즈', '뉴에이지', 'JPOP', '월드뮤직', 'CCM', '어린이/태교', '종교음악', '국악']

#에러는 따로 모아놓자
err_list = []

for idx in non_artist_num:
  url = main_url + Base_df['번호'].loc[idx]
  driver.get(url)
  time.sleep(3)

  try:
    body = driver.find_element_by_css_selector('body')
    res = driver.page_source
    soup = bs(res, "html.parser")

    #장르 업데이트
    print(Base_df['가수번호'].loc[idx], '->', end='')
    base_genre = Base_df['장르'].loc[idx].split(' ')
    if not (base_genre in genres):
      dl_list = soup.find('dl', {'class':'list'})
      dl_list_text = dl_list.text
      dl_list_text = dl_list_text.split('\n')
      genre_index = dl_list_text.index('장르')
      Base_df['장르'].loc[idx] = dl_list_text[genre_index+1]

    #노래에 추천 태그 긁어오기
    div_service_list_album = soup.find('div', {'class':'service_list_album'})
    tag_ul = div_service_list_album.find('ul')
    tag_div = tag_ul.find_all('div', {'class':'tag_list'})
    #긁어온 태그 리스트화
    tag_list = []
    for div in tag_div:
      tag_list += div.text.replace('\n', ' ')[1:-1].split(' ')
    #기존에 있던 태그 리스트화
    tag_list_ori = Base_df['태그'].loc[idx].split(' ')
    #기존태그 + 새로운태그 and set화하여 중복 제거 및 공백태그제거
    tag_list += tag_list_ori
    tag_set = set(tag_list)
    if '' in tag_set:
      tag_set.remove('')
    Base_df['태그'].loc[idx] = (' ').join(tag_set)

    artist_list = []
    #가수번호 긁어오기
    div_artist_list = soup.find('div', {'class':'artist'}).find_all('a')
    for div in div_artist_list:
      artist_list.append(div['href'].split("'")[-2])
    Base_df['가수번호'].loc[idx] = (' ').join(artist_list)
    #진행사항 표시
    print(Base_df['가수번호'].loc[idx], Base_df['장르'].loc[idx], Base_df['노래제목'].loc[idx], Base_df['태그'].loc[idx], Base_df['가수번호'].loc[idx], non_artist_num.index(idx)+1, '/', len(non_artist_num))
  except:
    print('err')
    err_list.append(idx)
    continue
#파일로 저장

print('error num', len(err_list))
Base_df.to_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/data/Base_Song_meta.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


 ->103289 랩/힙합 챔피언 (Feat. 정인) #랩 #맥주한잔과 #힙합 #센치 #감성 #감성힙합 #퇴근_후 #2000년대 #추억 #밤 103289 1 / 138


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


 ->406712 댄스 Fire #인기가요 #아이돌 #싸이월드 #댄스 #기분전환 #자존감 #스트레스 #업무 #2000년 #케이팝 #노동요 #과제 #2000년대 #마라맛 406712 2 / 138
 ->259822 댄스 Heartbeat #인기가요 #아이돌 #싸이월드 #댄스 #기분전환 #자존감 #스트레스 #업무 #2000년 #케이팝 #노동요 #과제 #2000년대 #마라맛 259822 3 / 138
 ->322666 댄스 뱅(Bang) ! #아이돌 #숨듣명 #댄스 #기분전환 #자존감 #스트레스 #업무 #노동요 #케이팝 #마라맛 #과제 #문명특급 #컴눈명 322666 4 / 138
 ->428501 댄스 NU 예삐오 (NU ABO) #댄스 #걸그룹 #기분전환 #자존감 #스트레스 #업무 #노동요 #케이팝 #마라맛 #과제 #아이돌 #추억 #발라드 428501 5 / 138
 ->2165 댄스 Hurricane Venus #숨듣명 #댄스 #기분전환 #자존감 #스트레스 #운동 #업무 #노동요 #케이팝 #마라맛 #과제 #아이돌 2165 6 / 138
 ->103289 랩/힙합 회상 (Feat. 백지영) #맥주한잔과 #힙합 #센치 #노래방 #감성 #감성힙합 #퇴근_후 #추억 #발라드 #밤 103289 7 / 138
 ->108358 랩/힙합 막잔하고 나갈게 (Lost One) #랩 #맥주한잔과 #힙합 #센치 #감성 #감성힙합 #퇴근_후 #추억 #밤 #회상 108358 8 / 138
 ->7560 랩/힙합 인생 (Feat. 웅산) #랩 #맥주한잔과 #힙합 #센치 #감성 #감성힙합 #퇴근_후 #추억 #발라드 #밤 7560 9 / 138
 ->532029 랩/힙합 If I Die Tomorrow #랩 #맥주한잔과 #힙합 #센치 #느낌있는 #드라이브 #감성 #감성힙합 #퇴근_후 #밤 532029 10 / 138
 ->35682 랩/힙합 3호선 매봉역 (Feat. Paloalto, Beenzino) #잔잔 #맥주한잔과 #노래추천 #힙합 #센치 #기분전환 #드라이브 #감성 #감성힙합 #

#가수번호 테스트

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/data/Base_Song_meta.csv')
df
df.isna().sum()


index     0
날짜        0
번호        0
앨범번호      0
앨범이름      1
가수       33
가수번호     33
노래제목      0
장르        0
좋아요       0
태그       33
dtype: int64

In [ ]:
df = df.fillna('')
df.loc[index]

,index,날짜,번호,앨범번호,앨범이름,가수,가수번호,노래제목,장르,좋아요,태그
3949,495599,20210714,495599,33187,The Black Album,JAY-Z,,Allure (Album Version (Explicit)),랩/힙합,140,#2021 #외국힙합 #신나는 #오바마 #팝 #여름 #기분전환 #명곡 #힙합
3950,624243,20210714,624243,43867,John Wesley Harding,Bob Dylan,,I'll Be Your Baby Tonight,포크,5,#2021 #신나는 #팝 #오바마 #여름 #기분전환 #명곡
3951,665230,20210714,665230,47037,Back To The Soul Flight,나얼,,호랑나비,R&B/Soul,"1,648",#추억 #충전 #인디 #발라드 #맑은 #상쾌 #그때그시절 #버스 #여름 #산책 #설...
3952,792716,20210714,792716,60671,Hejira,Joni Mitchell,,Coyote,포크,21,#2021 #신나는 #포크 #팝 #오바마 #여름 #레전드 #기분전환 #명곡
3953,1048246,20210714,1048246,311550,Exile On Main Street,The Rolling Stones,,Tumbling Dice,록/메탈,81,#블루스 #2021 #신나는 #오바마 #팝 #여름 #재즈 #기분전환 #명곡
...,...,...,...,...,...,...,...,...,...,...,...
4094,33649028,20210714,33649028,10644054,"Speak Your Mind (from the Netflix Series ""We T...",Brandi Carlile,,"Speak Your Mind (from the Netflix Series ""We T...","포크, 애니메이션/웹툰",2,#2021 #신나는 #팝 #오바마 #여름 #기분전환 #명곡
4095,33654984,20210714,33654984,10645397,Costa Rica,Benjamin Ingrosso,,Costa Rica,POP,24,#해안도로 #isfp #mbti #여름 #신나는 #드라이브 #휴가철
4096,106732663,20210714,106732663,5640898,Legend - The Best Of Bob Marley And The Wailers,Bob Marley & The Wailers,,Exodus,월드뮤직,5,#2021 #신나는 #팝 #오바마 #여름 #기분전환 #명곡
4097,106763707,20210714,106763707,5643772,Everybody Loves The Sunshine (Reissue),Roy Ayers Ubiquity,,Everybody Loves The Sunshine,재즈,108,#RnB #2021 #HipHop #신나는 #오바마 #팝 #여름 #기분전환 #명곡


In [ ]:
Base_df.to_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/data/Base_Song_meta.csv')

#테스트

In [ ]:
err_list

[32957554, 1556596, 1556597, 32978815]

In [ ]:
Base_df.loc[32978815]

날짜               20210708
번호               32978815
앨범번호             10501311
앨범이름          혜봉스님 불경 20집
가수                   혜봉스님
노래제목        천지팔양신주경 (우리말)
장르                종교음악 불교
좋아요                    12
태그      #인기순위 #일간순위 #종교음악
가수번호              2637644
Name: 32978815, dtype: object